In [1]:
import numpy as np
from collections import defaultdict, Counter
import math
import string
import re
from operator import itemgetter
from nltk.stem import PorterStemmer 
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import f1_score

In [61]:
# Definition of preprocessing functions

# Functional functions
def compose2(f, g):
    return lambda *a, **kw: f(g(*a, **kw))

def compose (*functions):
    def inner(arg):
        for f in reversed(functions):
            arg = f(arg)
        return arg
    return inner

# Filtering
has_valid_len = lambda w: len(w) >= 3
is_nonstop_word = lambda w: w not in ENGLISH_STOP_WORDS
is_valid_word = lambda w: has_valid_len(w) and is_nonstop_word(w)

# Mapping
to_lower = lambda w: w.lower()
remove_punc = lambda w: w.translate(str.maketrans('', '', string.punctuation))
remove_num = lambda w: re.sub(r'\d+', '', w)

map_word = compose(to_lower, remove_punc, remove_num)

# This mapping is expensive, so do it after filtering
ps = PorterStemmer() 
stem = lambda w: ps.stem(w)

def preprocess(docs):
    docs = [ [map_word(t) for t in d ] for d in docs ]
    docs = [ [t for t in d if is_valid_word(t)] for d in docs ]
    docs = [ [stem(t) for t in d ] for d in docs ]
    return docs

In [3]:
# Create sparse matrices, which requires aggregration of term IDs for both training and test data.

def calc_term_ids(docs):
    r""" The docs should be the combination of training set and test set."""
    term_ids = {}
    curr_term_id = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in term_ids:
                term_ids[w] = curr_term_id
                curr_term_id += 1
    return (term_ids, nnz)

def build_sparse_matrix(docs, term_ids = {}, nnz = 0):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    ncols = len(term_ids)
    assert(ncols != 0)

    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows + 1, dtype=np.int)
    row_id = 0  # document ID / row counter
    acc = 0  # non-zero counter

    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k, _ in cnt.most_common())
        curr_doc_len = len(keys)
        for i, key in enumerate(keys):
            ind[acc + i] = term_ids[key]
            val[acc + i] = cnt[key]
        ptr[row_id + 1] = ptr[row_id] + curr_doc_len
        acc += curr_doc_len
        row_id += 1

    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [4]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
    
def csr_normalize(mat, copy=False, **kargs):
    r""" combination of 'csr_idf' and 'csr_l2normalize' """
    if copy is True:
        mat = mat.copy()
    csr_idf(mat)
    csr_l2normalize(mat)
    if copy is True:
        return mat

In [43]:
def predict_w(test_data, train_data, labels, k=1):
    r""" Predict the label of test data based on the given
    labeled training data using k-nearest neighbor classifier.
    """
    predictions = []
    
    similarity = train_data.dot(test_data.T).todense()
    top_k_idx = np.argpartition(similarity, -k, axis=0)[-k:,:]
    to_labels = np.vectorize(lambda idx: labels[idx])
    top_k_label = to_labels(top_k_idx)
    
    n_test_col = similarity.shape[1]

    for col in range(n_test_col):
        train_tags = top_k_label[:,col].flatten().tolist()[0]
        train_indices = top_k_idx[:,col].flatten().tolist()[0]

        # Select the maximum aggregated similarity
        weights = defaultdict(float)
        for tag, row in zip(train_tags, train_indices):
            weights[tag] += similarity[row, col]
        result = max(weights.items(), key=itemgetter(1))[0]
        
        
 
        predictions.append(result)

    return predictions

In [6]:
def load_data_as_lines(path):
    r""" Open text file by path and read all lines """
    with open(path, "r") as fh:
        lines = fh.readlines()
        
    # transform docs into lists of words
    raw_lines = [l.split() for l in lines]

    return raw_lines

In [7]:
def to_data_and_label(raw_lines):
    r""" Split training data and label from raw lines """
    y = list(map(lambda x: int(x[0]), raw_lines))
    x = list(map(lambda x: x[1:], raw_lines))
    return (x, y)

In [51]:
def save_result_as_file(prediction, file_name="prediction.dat"):
    r""" Save the predicted result as a new file """
    file_content = "\n".join(list(map(str, prediction)))
    with open(file_name, "w") as fd:
        fd.write(file_content) 

In [10]:
def main():
    # Read files
    train_lines = load_data_as_lines("train.dat")
    x_test = load_data_as_lines("test.dat")
    
    # Split 
    x_train, y_train = to_data_and_label(train_lines)

    # Text preprocessing
    print("Preprocessing documents...")
    docs_train = preprocess(x_train)
    docs_test = preprocess(x_test)

    # Build shared term ID dictionary for classification
    print("Calculating terms...")
    term_ids, nnz = calc_term_ids([*docs_train, *docs_test])

    # Build sparse matrices
    print("Building sparse matrix...")
    mat_train = build_sparse_matrix(docs_train, term_ids, nnz)
    mat_test = build_sparse_matrix(docs_test, term_ids, nnz)

    # Normalize CSR matrices
    print("Normalizing matrix...")
    mat_test = csr_normalize(mat_test, copy=True)
    mat_train = csr_normalize(mat_train, copy=True)

    # Train
    print("Start training...")
    y_test = predict(mat_test, mat_train, y_train, k=6)
    save_result_as_file(y_test)
    print("Done!")

In [11]:
def partition(raw_list, folds):
    fold_size = math.floor(len(raw_list) / folds)
    train_set = []
    test_set = []
    for i in range(folds):
        if i == folds - 1:
            train_set.append(raw_list[:i*fold_size])
            test_set.append(raw_list[i*fold_size:])
        else:
            train_set.append(raw_list[:i*fold_size] + raw_list[(i+1)*fold_size:])
            test_set.append(raw_list[i*fold_size:(i+1)*fold_size])
    return train_set, test_set

def test(folds = 10, max_k = 500):
    r""" Testing with cross validation """
    
    print("Start testing mode...")
    scores = []

    # Read files
    train_lines = load_data_as_lines("train.dat")
    train_lines_set, test_lines_set = partition(train_lines, folds)
    
    train_lines_set = list(map(to_data_and_label, train_lines_set))
    test_lines_set = list(map(to_data_and_label, test_lines_set))
    
    for i in range(folds):
        print("> Running fold %d" % (i + 1))
        
        x_train, y_train = train_lines_set[i]
        x_test, y_test = test_lines_set[i]

        # Text preprocessing
        print("Preprocessing documents...")
        docs_train = preprocess(x_train)
        docs_test = preprocess(x_test)

        # Build shared term ID dictionary for classification
        print("Calculating terms...")
        term_ids, nnz = calc_term_ids([*docs_train, *docs_test])

        # Build sparse matrices
        print("Building sparse matrix...")
        mat_train = build_sparse_matrix(docs_train, term_ids, nnz)
        mat_test = build_sparse_matrix(docs_test, term_ids, nnz)

        # Normalize CSR matrices
        print("Normalizing matrix...")
        mat_test = csr_normalize(mat_test, copy=True)
        mat_train = csr_normalize(mat_train, copy=True)

        # Prediction
        for k in range(max_k):
            y_hat = predict(mat_test, mat_train, y_train, k=k)
            scores.append(f1_score(y_test, y_hat, average='macro'))
            
    avg_score = np.average(scores, axis=1)
    res = [{"k": i + 1, "score": avg_score[i]} for i in range(len(avg_score))]

    return sorted(res, key=lambda x: x["score"], reverse=True)

In [12]:
if __name__ == "__main__":
    main()

Preprocessing documents...
Calculating terms...
Building sparse matrix...
Normalizing matrix...
Start training...
Done!
